In [1]:
import os
import numpy as np
import pandas as pd
import pickle


In [2]:
os.listdir('/mnt/hdd0/qllai/cluster/')

['All_pro2ec_save.pkl',
 'nohup_corpus.out',
 'All_pro2ec_mibig.pkl',
 'corpus_mibig.txt',
 '.ipynb_checkpoints',
 'nohup_mibig.out',
 'Unknown_mibig.pkl',
 'pool_outputs.pkl',
 'Cluster.ipynb',
 'generate_corpus_save.py',
 'nohup.out.bak',
 'test.py',
 'generate_corpus.py',
 'pool_outputs_mibig.pkl',
 'Generate_BGC_Train_dataset.ipynb',
 'concat_no_BGC_corpus.txt',
 'nohup.out',
 'pool_outputs_save.pkl',
 'generate_corpus_mibig.py',
 'Unknown_save.pkl',
 'nohup.out.bak.bak',
 'getPositive.py',
 'BGC2line.pkl',
 'corpus.txt']

In [3]:
with open('/mnt/hdd0/qllai/cluster/concat_no_BGC_corpus.txt', 'r') as fp:
    concated_no_BGC_corpus = fp.read()
    concated_no_BGC_corpus = concated_no_BGC_corpus.split()
    print(len(concated_no_BGC_corpus))

74158968


In [4]:
os.listdir()

['BGC_labels_dataset.csv',
 'NineGm_TD_origin_dataset.csv',
 'NineGm_TD_Augmentation_dataset.csv',
 'NineGenomes.csv',
 'gensim_word2id.pkl',
 'BGC_TD_dataset_positive.csv',
 'BGC_labels_eval_dataset.csv',
 'positiveSamples.py',
 'BGC_labels_dataset_positive.csv',
 'gensim_id2word.pkl',
 'word_frequency_dic_min3.pkl',
 'get_train_dataset_genome_insert.ipynb',
 'BGC_labels_dataset.pkl',
 'NineGenomes_128.csv',
 'mibig_labels.pkl',
 'corpus_word2vec.vector',
 'mibig_word_frequency_dic.pkl',
 'getNegativeSamples.py',
 'corpus_word2vec.py',
 'sampling_chi2.py',
 'corpus_word2vec.sav.syn1neg.npy',
 'imgABC_augmentation.csv',
 'imgABC_origin_data.csv',
 'corpus_word2vec.sav.wv.vectors.npy',
 'Sampling_chi2_2.png',
 'NineGm_TD_testing_positive_dataset.csv',
 'testing_TD_dataset.csv',
 'BGC_TD_dataset_negative.csv',
 'bgc2idseq.csv',
 '9genomes_testing_data.txt',
 'corpus_word2vec.sav',
 'BGC_TD_dataset.csv',
 'BGC_TD_dataset_balance.csv',
 'NonBGC_TD_testing_dataset_negative.csv',
 'BGC2line.

In [12]:
BGC_labels_dataset = pd.read_csv('./BGC_labels_dataset.csv')
BGC_labels_dataset

,ID,sentence,labels,isBGC
0,BGC0000436,Cluster_1153963 Cluster_1501678 Cluster_160734...,NRP,Yes
1,BGC0000918,Cluster_0698230 EC:1.13.11.27 Cluster_1230425 ...,Other,Yes
2,BGC0000740,Cluster_1607346 Cluster_0676609 Cluster_090172...,Saccharide,Yes
3,BGC0002659,Cluster_1042555 Cluster_0095569 Cluster_176389...,RiPP,Yes
4,BGC0001750,Cluster_0175250 Cluster_1313057 EC:6.2.1.3 Clu...,NRP Polyketide,Yes
...,...,...,...,...
10003,HalfMibigHalfGenome_7501,EC:5.3.3.2 Cluster_1049777 Cluster_1641406 Clu...,Non,No
10004,HalfMibigHalfGenome_7502,Cluster_1322604 EC:6.2.1.3 Cluster_0864922 EC:...,Non,No
10005,HalfMibigHalfGenome_7503,Cluster_0513799 Cluster_1249665 Cluster_1682136,Non,No
10006,HalfMibigHalfGenome_7504,Cluster_1843822 Cluster_0940597 Cluster_032120...,Non,No


In [9]:
BGC_labels_dataset.iloc[2502]

ID                                      HalfMibigHalfGenome_0
sentence    Cluster_1760951 Cluster_1044407 Cluster_043984...
labels                                                    Non
isBGC                                                      No
Name: 2502, dtype: object

In [10]:
BGC_origin_dataset = BGC_labels_dataset[:2502]

In [13]:
BGC_origin_dataset[BGC_origin_dataset['isBGC']!='Yes']

,ID,sentence,labels,isBGC


In [14]:
import random
def generateInsertList(num, maxLen=64, notInsertProb:float=0.5, maxNonBGCgenes:int=8):
    '''
    生成插入到序列每个空位非BGC的基因个数，首尾也插入，有插入概率与长度的限制
    num: BGC序列长度
    notInsertPro: 该空位不插入非BGC基因的概率，此处为概率而非比例，有可能每个空位都插入也有可能每个空位均不插入
    maxNonBGCgenes: 一个空位插入非BGC基因的最大长度
    '''
    randomNonBGClen = list(range(0, maxNonBGCgenes+1))
    randomNonBGClenWeight = [notInsertProb, *[(1-notInsertProb)/maxNonBGCgenes]*maxNonBGCgenes]
    randomLen = random.choices(population=randomNonBGClen, weights=randomNonBGClenWeight, k=num-1)
    randomStart, randomEnd = [0], [0]
    leftLen = maxLen-num-sum(randomLen)
    randomStart = [0]
    randomEnd = [0]
    if leftLen>0:
        randomStart = random.randint(0, leftLen)
        randomEnd = [leftLen - randomStart]
        randomStart = [randomStart]
    return randomStart + randomLen + randomEnd
a = generateInsertList(20, 128, 0.8, 5)
print(a)

[13, 0, 0, 0, 0, 5, 0, 0, 3, 0, 2, 1, 3, 0, 0, 0, 0, 0, 0, 0, 81]


In [16]:
len_corpus = len(concated_no_BGC_corpus)
concated_no_BGC_corpus_plus = concated_no_BGC_corpus + concated_no_BGC_corpus[:128]

In [17]:
print(len(concated_no_BGC_corpus))
print(len(concated_no_BGC_corpus_plus))

74158968
74159096


In [23]:
def getOneAugSeq(seq, maxLen, notInsertProb=0.8, maxNonBGCgenes=5):
    num = len(seq)
    insertList = generateInsertList(num=num, maxLen=maxLen, notInsertProb=notInsertProb, maxNonBGCgenes=maxNonBGCgenes)
    # print(insertList)
    random_start = random.randint(0, len_corpus)
    random_end = random_start + maxLen - num
    random_genes = concated_no_BGC_corpus_plus[random_start:random_end]
    trainSeq = []
    trainSeqLabels = []
    for i in range(len(insertList)):
        insert_seq_num = insertList[i]
        insert_seq = random_genes[:insert_seq_num]
        random_genes = random_genes[insert_seq_num:]
        trainSeq += insert_seq
        trainSeqLabels += [0]*insert_seq_num
        if i<num:
            trainSeq += [seq[i]]
            trainSeqLabels += [1]
    trainSeq = trainSeq[:maxLen]
    trainSeqLabels = trainSeqLabels[:maxLen]
    return trainSeq, trainSeqLabels
trainSeq, trainSeqLabels = getOneAugSeq(['Cluster_1153963', 'Cluster_1501678', 'Cluster_1607345', 'Cluster_1534836', 'Cluster_1124421', 'EC:6.2.1.3', 'Cluster_0034481'], maxLen=128)
print(trainSeq)
print(trainSeqLabels)

['Cluster_1438111', 'Cluster_1199491', 'Cluster_1871582', 'Cluster_0162748', 'Cluster_1459643', 'Cluster_0237303', 'Cluster_0354626', 'Cluster_1848395', 'Cluster_0271927', 'Cluster_0090990', 'Cluster_1273927', 'Cluster_0964353', 'Cluster_1065505', 'Cluster_0532285', 'Cluster_0872812', 'Cluster_1152145', 'Cluster_0357776', 'Cluster_1899467', 'Cluster_0643898', 'Cluster_1074174', 'Cluster_0308738', 'Cluster_0228500', 'Cluster_0359315', 'Cluster_0047994', 'Cluster_1258638', 'Cluster_0074952', 'Cluster_0414164', 'Cluster_0249449', 'Cluster_0615989', 'Cluster_0783859', 'Cluster_1570340', 'Cluster_1596802', 'Cluster_1608114', 'Cluster_0646121', 'Cluster_0594441', 'Cluster_1504199', 'Cluster_0830206', 'Cluster_1185979', 'Cluster_0097941', 'Cluster_1878129', 'Cluster_0432850', 'Cluster_0650519', 'Cluster_0961811', 'Cluster_1879907', 'Cluster_0124443', 'Cluster_0945257', 'Cluster_0659225', 'Cluster_1773378', 'Cluster_0726125', 'Cluster_0012624', 'Cluster_0009924', 'Cluster_1028688', 'Cluster_08

In [22]:
def getOneOriginSeq(seq, maxLen):
    num = len(seq)
    trainSeq = seq[:]
    trainSeqLabels = [1]*num
    random_start = random.randint(0, len_corpus)
    random_end = random_start + maxLen - num
    random_genes = concated_no_BGC_corpus_plus[random_start:random_end]
    trainSeq += random_genes
    trainSeqLabels += [0] * (maxLen-num)
    return trainSeq, trainSeqLabels
trainSeq, trainSeqLabels = getOneOriginSeq(['Cluster_1153963', 'Cluster_1501678', 'Cluster_1607345', 'Cluster_1534836', 'Cluster_1124421', 'EC:6.2.1.3', 'Cluster_0034481'], maxLen=128)
print(trainSeq)
print(trainSeqLabels)

['Cluster_1153963', 'Cluster_1501678', 'Cluster_1607345', 'Cluster_1534836', 'Cluster_1124421', 'EC:6.2.1.3', 'Cluster_0034481', 'Cluster_0912977', 'Cluster_0169864', 'Cluster_1481278', 'Cluster_1443407', 'Cluster_0115036', 'Cluster_1070950', 'EC:1.8.4.11', 'Cluster_0933664', 'Cluster_0038767', 'Cluster_1369783', 'Cluster_0722786', 'Cluster_0874878', 'Cluster_1892640', 'Cluster_1745256', 'EC:6.1.1.14', 'Cluster_1642975', 'Cluster_0066109', 'EC:4.1.3.3', 'Cluster_1541733', 'EC:4.1.2.13', 'Cluster_1022876', 'EC:1.1.1.27', 'Cluster_0434458', 'Cluster_0784746', 'Cluster_1740846', 'Cluster_1049870', 'Cluster_0286518', 'Cluster_1872666', 'Cluster_0989689', 'Cluster_1340728', 'Cluster_1740640', 'Cluster_1603680', 'Cluster_1558083', 'Cluster_1340728', 'Cluster_1523610', 'Cluster_0317305', 'Cluster_0194213', 'Cluster_1170990', 'Cluster_1104976', 'Cluster_1104976', 'Cluster_1104976', 'Cluster_1104976', 'Cluster_1660922', 'Cluster_1104976', 'Cluster_1104976', 'EC:2.7.1.113', 'EC:2.4.2.10', 'Clust

In [25]:
random.seed(42)
def generatePositiveSamples(dataset, maxLen, notInsertProb, maxNonBGCgenes, times):
    BGC_postive_dataset = pd.DataFrame(columns=['ID', 'sentence', 'labels', 'isBGC', 'TDsentence', 'TDlabels'])
    for i in range(len(dataset)):
        ID = dataset.iloc[i]['ID']
        seq = dataset.iloc[i]['sentence']
        labels = dataset.iloc[i]['labels']
        isBGC = dataset.iloc[i]['isBGC']
        trainSeq, trainSeqLabels = getOneOriginSeq(seq=seq, maxLen=maxLen)
        BGC_postive_dataset.loc[len(BGC_postive_dataset)] = [ID+'_0',seq, labels, isBGC, trainSeq, trainSeqLabels]

        for j in range(times):
            trainSeq, trainSeqLabels = getOneAugSeq(seq=seq, maxLen=maxLen, notInsertProb=notInsertProb, maxNonBGCgenes=maxNonBGCgenes)
            BGC_postive_dataset.loc[len(BGC_postive_dataset)] = [ID+f'_{j+1}',seq, labels, isBGC, trainSeq, trainSeqLabels]
    return BGC_postive_dataset


In [28]:
BGC_origin_dataset['sentence'] = BGC_origin_dataset['sentence'].apply(lambda x:x.split())
BGC_origin_dataset

/tmp/ipykernel_1310844/3702316407.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  BGC_origin_dataset['sentence'] = BGC_origin_dataset['sentence'].apply(lambda x:x.split())


,ID,sentence,labels,isBGC
0,BGC0000436,"[Cluster_1153963, Cluster_1501678, Cluster_160...",NRP,Yes
1,BGC0000918,"[Cluster_0698230, EC:1.13.11.27, Cluster_12304...",Other,Yes
2,BGC0000740,"[Cluster_1607346, Cluster_0676609, Cluster_090...",Saccharide,Yes
3,BGC0002659,"[Cluster_1042555, Cluster_0095569, Cluster_176...",RiPP,Yes
4,BGC0001750,"[Cluster_0175250, Cluster_1313057, EC:6.2.1.3,...",NRP Polyketide,Yes
...,...,...,...,...
2497,BGC0000177,"[Cluster_1377462, Cluster_0008689, Cluster_062...",Polyketide,Yes
2498,BGC0000892,"[Cluster_0385013, Cluster_1806792, Cluster_045...",Other,Yes
2499,BGC0001718,"[EC:2.7.7.48, Cluster_1722574, Cluster_0431146...",NRP,Yes
2500,BGC0000213,"[Cluster_0417466, Cluster_1653936, Cluster_156...",Polyketide,Yes


In [29]:
BGC_postive_dataset = generatePositiveSamples(dataset=BGC_origin_dataset, maxLen=128, notInsertProb=0.8, maxNonBGCgenes=5, times=10)
BGC_postive_dataset

,ID,sentence,labels,isBGC,TDsentence,TDlabels
0,BGC0000436_0,"[Cluster_1153963, Cluster_1501678, Cluster_160...",NRP,Yes,"[Cluster_1153963, Cluster_1501678, Cluster_160...","[1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,BGC0000436_1,"[Cluster_1153963, Cluster_1501678, Cluster_160...",NRP,Yes,"[Cluster_0876677, Cluster_0442068, Cluster_093...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,BGC0000436_2,"[Cluster_1153963, Cluster_1501678, Cluster_160...",NRP,Yes,"[EC:3.4.13.9, Cluster_0849311, Cluster_1331855...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,BGC0000436_3,"[Cluster_1153963, Cluster_1501678, Cluster_160...",NRP,Yes,"[Cluster_1629811, Cluster_1297778, Cluster_149...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,BGC0000436_4,"[Cluster_1153963, Cluster_1501678, Cluster_160...",NRP,Yes,"[Cluster_1621985, Cluster_1595642, Cluster_157...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...,...,...,...
27517,BGC0001595_6,"[Cluster_0923528, Cluster_1221717, Cluster_030...",Alkaloid,Yes,"[Cluster_1857686, Cluster_1652984, Cluster_023...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
27518,BGC0001595_7,"[Cluster_0923528, Cluster_1221717, Cluster_030...",Alkaloid,Yes,"[Cluster_1128388, Cluster_1603606, Cluster_003...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
27519,BGC0001595_8,"[Cluster_0923528, Cluster_1221717, Cluster_030...",Alkaloid,Yes,"[Cluster_0322584, Cluster_1434017, Cluster_114...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, ..."
27520,BGC0001595_9,"[Cluster_0923528, Cluster_1221717, Cluster_030...",Alkaloid,Yes,"[Cluster_1349909, Cluster_0931083, Cluster_017...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [35]:
BGC_postive_dataset.iloc[[0,11,22,33]]

,ID,sentence,labels,isBGC,TDsentence,TDlabels
0,BGC0000436_0,"[Cluster_1153963, Cluster_1501678, Cluster_160...",NRP,Yes,"[Cluster_1153963, Cluster_1501678, Cluster_160...","[1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, ..."
11,BGC0000918_0,"[Cluster_0698230, EC:1.13.11.27, Cluster_12304...",Other,Yes,"[Cluster_0698230, EC:1.13.11.27, Cluster_12304...","[1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
22,BGC0000740_0,"[Cluster_1607346, Cluster_0676609, Cluster_090...",Saccharide,Yes,"[Cluster_1607346, Cluster_0676609, Cluster_090...","[1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, ..."
33,BGC0002659_0,"[Cluster_1042555, Cluster_0095569, Cluster_176...",RiPP,Yes,"[Cluster_1042555, Cluster_0095569, Cluster_176...","[1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [36]:
origin_index = [i*11 for i in range(len(BGC_origin_dataset))]
five_times_index_ = [[i*11+1,i*11+2,i*11+3,i*11+4,i*11+5] for i in range(len(BGC_origin_dataset))]
five_times_index = []
for l in five_times_index_:
    five_times_index += l

In [38]:
origin_index[:20]

[0,
 11,
 22,
 33,
 44,
 55,
 66,
 77,
 88,
 99,
 110,
 121,
 132,
 143,
 154,
 165,
 176,
 187,
 198,
 209]

In [39]:
five_times_index[:30]

[1,
 2,
 3,
 4,
 5,
 12,
 13,
 14,
 15,
 16,
 23,
 24,
 25,
 26,
 27,
 34,
 35,
 36,
 37,
 38,
 45,
 46,
 47,
 48,
 49,
 56,
 57,
 58,
 59,
 60]

In [51]:
BGC_postive_dataset_5 = BGC_postive_dataset.iloc[origin_index+five_times_index]
BGC_postive_dataset_5 = BGC_postive_dataset_5.reset_index()
BGC_postive_dataset_5

,index,ID,sentence,labels,isBGC,TDsentence,TDlabels
0,0,BGC0000436_0,"[Cluster_1153963, Cluster_1501678, Cluster_160...",NRP,Yes,"[Cluster_1153963, Cluster_1501678, Cluster_160...","[1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,11,BGC0000918_0,"[Cluster_0698230, EC:1.13.11.27, Cluster_12304...",Other,Yes,"[Cluster_0698230, EC:1.13.11.27, Cluster_12304...","[1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,22,BGC0000740_0,"[Cluster_1607346, Cluster_0676609, Cluster_090...",Saccharide,Yes,"[Cluster_1607346, Cluster_0676609, Cluster_090...","[1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,33,BGC0002659_0,"[Cluster_1042555, Cluster_0095569, Cluster_176...",RiPP,Yes,"[Cluster_1042555, Cluster_0095569, Cluster_176...","[1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,44,BGC0001750_0,"[Cluster_0175250, Cluster_1313057, EC:6.2.1.3,...",NRP Polyketide,Yes,"[Cluster_0175250, Cluster_1313057, EC:6.2.1.3,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
...,...,...,...,...,...,...,...
15007,27512,BGC0001595_1,"[Cluster_0923528, Cluster_1221717, Cluster_030...",Alkaloid,Yes,"[Cluster_1303400, Cluster_0216053, Cluster_096...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
15008,27513,BGC0001595_2,"[Cluster_0923528, Cluster_1221717, Cluster_030...",Alkaloid,Yes,"[Cluster_1829793, Cluster_0352190, Cluster_000...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
15009,27514,BGC0001595_3,"[Cluster_0923528, Cluster_1221717, Cluster_030...",Alkaloid,Yes,"[Cluster_0874858, Cluster_1816892, Cluster_074...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
15010,27515,BGC0001595_4,"[Cluster_0923528, Cluster_1221717, Cluster_030...",Alkaloid,Yes,"[Cluster_0923528, Cluster_1221717, EC:3.5.1.96...","[1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, ..."


In [46]:
BGC_postive_dataset_5['sentence'] = BGC_postive_dataset_5['sentence'].apply(lambda x:' '.join(x))
BGC_postive_dataset_5['TDsentence'] = BGC_postive_dataset_5['TDsentence'].apply(lambda x:' '.join(x))
BGC_postive_dataset_5['TDlabels'] = BGC_postive_dataset_5['TDlabels'].apply(lambda x:[str(_) for _ in x]).apply(lambda x:' '.join(x))
BGC_postive_dataset_5

,index,ID,sentence,labels,isBGC,TDsentence,TDlabels
0,0,BGC0000436_0,Cluster_1153963 Cluster_1501678 Cluster_160734...,NRP,Yes,Cluster_1153963 Cluster_1501678 Cluster_160734...,1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
1,11,BGC0000918_0,Cluster_0698230 EC:1.13.11.27 Cluster_1230425 ...,Other,Yes,Cluster_0698230 EC:1.13.11.27 Cluster_1230425 ...,1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
2,22,BGC0000740_0,Cluster_1607346 Cluster_0676609 Cluster_090172...,Saccharide,Yes,Cluster_1607346 Cluster_0676609 Cluster_090172...,1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
3,33,BGC0002659_0,Cluster_1042555 Cluster_0095569 Cluster_176389...,RiPP,Yes,Cluster_1042555 Cluster_0095569 Cluster_176389...,1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
4,44,BGC0001750_0,Cluster_0175250 Cluster_1313057 EC:6.2.1.3 Clu...,NRP Polyketide,Yes,Cluster_0175250 Cluster_1313057 EC:6.2.1.3 Clu...,1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 ...
...,...,...,...,...,...,...,...
15007,27512,BGC0001595_1,Cluster_0923528 Cluster_1221717 Cluster_030676...,Alkaloid,Yes,Cluster_1303400 Cluster_0216053 Cluster_096092...,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
15008,27513,BGC0001595_2,Cluster_0923528 Cluster_1221717 Cluster_030676...,Alkaloid,Yes,Cluster_1829793 Cluster_0352190 Cluster_000140...,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
15009,27514,BGC0001595_3,Cluster_0923528 Cluster_1221717 Cluster_030676...,Alkaloid,Yes,Cluster_0874858 Cluster_1816892 Cluster_074327...,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 ...
15010,27515,BGC0001595_4,Cluster_0923528 Cluster_1221717 Cluster_030676...,Alkaloid,Yes,Cluster_0923528 Cluster_1221717 EC:3.5.1.96 Cl...,1 1 0 0 0 0 0 1 0 0 0 1 1 0 0 0 0 1 0 1 1 1 1 ...


In [47]:
BGC_postive_dataset_5.to_csv('./BGC_positve_dataset_5.csv')

In [49]:

def getNegative(maxLen):
    random_start = random.randint(0, len_corpus)
    random_end = random_start + maxLen
    random_genes = concated_no_BGC_corpus_plus[random_start:random_end]
    trainSeq = random_genes[:]
    trainSeqLabels = [0] * maxLen
    return trainSeq, trainSeqLabels
trainSeq, trainSeqLabels = getNegative(maxLen=128)
print(trainSeq)
print(trainSeqLabels)

['Cluster_0686210', 'Cluster_0912731', 'Cluster_0872946', 'Cluster_0994414', 'Cluster_1437433', 'Cluster_1173849', 'Cluster_0235810', 'Cluster_0360198', 'Cluster_0758189', 'Cluster_0018760', 'Cluster_0927257', 'Cluster_1523366', 'Cluster_0083837', 'Cluster_1330139', 'Cluster_1715094', 'Cluster_0744814', 'Cluster_0686202', 'Cluster_1445075', 'Cluster_1204982', 'EC:2.1.1.113', 'Cluster_1715094', 'Cluster_0953274', 'Cluster_1843261', 'Cluster_1173849', 'Cluster_0617660', 'EC:2.7.4.6', 'Cluster_0699048', 'EC:3.5.1.108', 'Cluster_0524512', 'Cluster_0916525', 'Cluster_0097845', 'Cluster_1242087', 'Cluster_0519486', 'Cluster_1099937', 'Cluster_0524606', 'Cluster_1192964', 'Cluster_0924785', 'Cluster_0111548', 'Cluster_0651382', 'Cluster_1694042', 'Cluster_0998175', 'Cluster_1613944', 'Cluster_0617659', 'Cluster_0308713', 'Cluster_1758789', 'Cluster_1768318', 'Cluster_0454855', 'Cluster_1430770', 'Cluster_1372903', 'Cluster_0652829', 'Cluster_1265375', 'Cluster_1303332', 'Cluster_0054067', 'Cl

In [50]:
BGC_negative_dataset = pd.DataFrame(columns=['ID', 'sentence', 'labels', 'isBGC', 'TDsentence', 'TDlabels'])
for i in range(len(BGC_postive_dataset)):
    trainSeq, trainSeqLabels = getNegative(maxLen=128)
    BGC_negative_dataset.loc[len(BGC_negative_dataset)] = [f'Negative_{i}',trainSeq, 'Unknown', 'No', trainSeq, trainSeqLabels]
BGC_negative_dataset

,ID,sentence,labels,isBGC,TDsentence,TDlabels
0,Negative_0,"[Cluster_0015572, Cluster_1420348, Cluster_097...",Unknown,No,"[Cluster_0015572, Cluster_1420348, Cluster_097...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,Negative_1,"[Cluster_1297103, Cluster_1594823, Cluster_180...",Unknown,No,"[Cluster_1297103, Cluster_1594823, Cluster_180...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,Negative_2,"[Cluster_1265438, Cluster_1223821, Cluster_173...",Unknown,No,"[Cluster_1265438, Cluster_1223821, Cluster_173...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,Negative_3,"[Cluster_1565322, Cluster_1734665, Cluster_014...",Unknown,No,"[Cluster_1565322, Cluster_1734665, Cluster_014...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,Negative_4,"[Cluster_0405709, Cluster_0837337, Cluster_077...",Unknown,No,"[Cluster_0405709, Cluster_0837337, Cluster_077...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...,...,...,...
27517,Negative_27517,"[Cluster_0610654, Cluster_1497579, Cluster_099...",Unknown,No,"[Cluster_0610654, Cluster_1497579, Cluster_099...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
27518,Negative_27518,"[Cluster_0000170, Cluster_1620477, EC:3.2.1.19...",Unknown,No,"[Cluster_0000170, Cluster_1620477, EC:3.2.1.19...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
27519,Negative_27519,"[Cluster_0525070, Cluster_0311313, Cluster_023...",Unknown,No,"[Cluster_0525070, Cluster_0311313, Cluster_023...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
27520,Negative_27520,"[Cluster_0130209, Cluster_1286778, EC:3.4.11.4...",Unknown,No,"[Cluster_0130209, Cluster_1286778, EC:3.4.11.4...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [58]:
BGC_TD_dataset_5 = pd.concat([BGC_postive_dataset_5, BGC_negative_dataset[:len(BGC_postive_dataset_5)]], ignore_index=True)
BGC_TD_dataset_5

,ID,sentence,labels,isBGC,TDsentence,TDlabels
0,BGC0000436_0,"[Cluster_1153963, Cluster_1501678, Cluster_160...",NRP,Yes,"[Cluster_1153963, Cluster_1501678, Cluster_160...","[1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,BGC0000918_0,"[Cluster_0698230, EC:1.13.11.27, Cluster_12304...",Other,Yes,"[Cluster_0698230, EC:1.13.11.27, Cluster_12304...","[1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,BGC0000740_0,"[Cluster_1607346, Cluster_0676609, Cluster_090...",Saccharide,Yes,"[Cluster_1607346, Cluster_0676609, Cluster_090...","[1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,BGC0002659_0,"[Cluster_1042555, Cluster_0095569, Cluster_176...",RiPP,Yes,"[Cluster_1042555, Cluster_0095569, Cluster_176...","[1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,BGC0001750_0,"[Cluster_0175250, Cluster_1313057, EC:6.2.1.3,...",NRP Polyketide,Yes,"[Cluster_0175250, Cluster_1313057, EC:6.2.1.3,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
...,...,...,...,...,...,...
30019,Negative_15007,"[Cluster_0511072, Cluster_0355445, Cluster_048...",Unknown,No,"[Cluster_0511072, Cluster_0355445, Cluster_048...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
30020,Negative_15008,"[Cluster_1642682, Cluster_1280940, Cluster_176...",Unknown,No,"[Cluster_1642682, Cluster_1280940, Cluster_176...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
30021,Negative_15009,"[Cluster_0408670, Cluster_1466465, Cluster_063...",Unknown,No,"[Cluster_0408670, Cluster_1466465, Cluster_063...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
30022,Negative_15010,"[Cluster_1302719, Cluster_1374535, Cluster_130...",Unknown,No,"[Cluster_1302719, Cluster_1374535, Cluster_130...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [57]:
BGC_postive_dataset_5 = BGC_postive_dataset_5.drop(columns='index')
BGC_postive_dataset_5

,ID,sentence,labels,isBGC,TDsentence,TDlabels
0,BGC0000436_0,"[Cluster_1153963, Cluster_1501678, Cluster_160...",NRP,Yes,"[Cluster_1153963, Cluster_1501678, Cluster_160...","[1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,BGC0000918_0,"[Cluster_0698230, EC:1.13.11.27, Cluster_12304...",Other,Yes,"[Cluster_0698230, EC:1.13.11.27, Cluster_12304...","[1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,BGC0000740_0,"[Cluster_1607346, Cluster_0676609, Cluster_090...",Saccharide,Yes,"[Cluster_1607346, Cluster_0676609, Cluster_090...","[1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,BGC0002659_0,"[Cluster_1042555, Cluster_0095569, Cluster_176...",RiPP,Yes,"[Cluster_1042555, Cluster_0095569, Cluster_176...","[1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,BGC0001750_0,"[Cluster_0175250, Cluster_1313057, EC:6.2.1.3,...",NRP Polyketide,Yes,"[Cluster_0175250, Cluster_1313057, EC:6.2.1.3,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
...,...,...,...,...,...,...
15007,BGC0001595_1,"[Cluster_0923528, Cluster_1221717, Cluster_030...",Alkaloid,Yes,"[Cluster_1303400, Cluster_0216053, Cluster_096...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
15008,BGC0001595_2,"[Cluster_0923528, Cluster_1221717, Cluster_030...",Alkaloid,Yes,"[Cluster_1829793, Cluster_0352190, Cluster_000...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
15009,BGC0001595_3,"[Cluster_0923528, Cluster_1221717, Cluster_030...",Alkaloid,Yes,"[Cluster_0874858, Cluster_1816892, Cluster_074...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
15010,BGC0001595_4,"[Cluster_0923528, Cluster_1221717, Cluster_030...",Alkaloid,Yes,"[Cluster_0923528, Cluster_1221717, EC:3.5.1.96...","[1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, ..."


In [59]:
BGC_TD_dataset_5['sentence'] = BGC_TD_dataset_5['sentence'].apply(lambda x:' '.join(x))
BGC_TD_dataset_5['TDsentence'] = BGC_TD_dataset_5['TDsentence'].apply(lambda x:' '.join(x))
BGC_TD_dataset_5['TDlabels'] = BGC_TD_dataset_5['TDlabels'].apply(lambda x:[str(_) for _ in x]).apply(lambda x:' '.join(x))
BGC_TD_dataset_5

,ID,sentence,labels,isBGC,TDsentence,TDlabels
0,BGC0000436_0,Cluster_1153963 Cluster_1501678 Cluster_160734...,NRP,Yes,Cluster_1153963 Cluster_1501678 Cluster_160734...,1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
1,BGC0000918_0,Cluster_0698230 EC:1.13.11.27 Cluster_1230425 ...,Other,Yes,Cluster_0698230 EC:1.13.11.27 Cluster_1230425 ...,1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
2,BGC0000740_0,Cluster_1607346 Cluster_0676609 Cluster_090172...,Saccharide,Yes,Cluster_1607346 Cluster_0676609 Cluster_090172...,1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
3,BGC0002659_0,Cluster_1042555 Cluster_0095569 Cluster_176389...,RiPP,Yes,Cluster_1042555 Cluster_0095569 Cluster_176389...,1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
4,BGC0001750_0,Cluster_0175250 Cluster_1313057 EC:6.2.1.3 Clu...,NRP Polyketide,Yes,Cluster_0175250 Cluster_1313057 EC:6.2.1.3 Clu...,1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 ...
...,...,...,...,...,...,...
30019,Negative_15007,Cluster_0511072 Cluster_0355445 Cluster_048772...,Unknown,No,Cluster_0511072 Cluster_0355445 Cluster_048772...,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
30020,Negative_15008,Cluster_1642682 Cluster_1280940 Cluster_176401...,Unknown,No,Cluster_1642682 Cluster_1280940 Cluster_176401...,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
30021,Negative_15009,Cluster_0408670 Cluster_1466465 Cluster_063779...,Unknown,No,Cluster_0408670 Cluster_1466465 Cluster_063779...,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
30022,Negative_15010,Cluster_1302719 Cluster_1374535 Cluster_130044...,Unknown,No,Cluster_1302719 Cluster_1374535 Cluster_130044...,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...


In [68]:
BGC_TD_dataset_5.to_csv('./BGC_TD_dataset_5.csv')

In [63]:
BGC_TD_dataset_10 = pd.concat([BGC_postive_dataset, BGC_negative_dataset], ignore_index=True)
BGC_TD_dataset_10

,ID,sentence,labels,isBGC,TDsentence,TDlabels
0,BGC0000436_0,"[Cluster_1153963, Cluster_1501678, Cluster_160...",NRP,Yes,"[Cluster_1153963, Cluster_1501678, Cluster_160...","[1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,BGC0000436_1,"[Cluster_1153963, Cluster_1501678, Cluster_160...",NRP,Yes,"[Cluster_0876677, Cluster_0442068, Cluster_093...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,BGC0000436_2,"[Cluster_1153963, Cluster_1501678, Cluster_160...",NRP,Yes,"[EC:3.4.13.9, Cluster_0849311, Cluster_1331855...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,BGC0000436_3,"[Cluster_1153963, Cluster_1501678, Cluster_160...",NRP,Yes,"[Cluster_1629811, Cluster_1297778, Cluster_149...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,BGC0000436_4,"[Cluster_1153963, Cluster_1501678, Cluster_160...",NRP,Yes,"[Cluster_1621985, Cluster_1595642, Cluster_157...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...,...,...,...
55039,Negative_27517,"[Cluster_0610654, Cluster_1497579, Cluster_099...",Unknown,No,"[Cluster_0610654, Cluster_1497579, Cluster_099...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
55040,Negative_27518,"[Cluster_0000170, Cluster_1620477, EC:3.2.1.19...",Unknown,No,"[Cluster_0000170, Cluster_1620477, EC:3.2.1.19...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
55041,Negative_27519,"[Cluster_0525070, Cluster_0311313, Cluster_023...",Unknown,No,"[Cluster_0525070, Cluster_0311313, Cluster_023...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
55042,Negative_27520,"[Cluster_0130209, Cluster_1286778, EC:3.4.11.4...",Unknown,No,"[Cluster_0130209, Cluster_1286778, EC:3.4.11.4...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [64]:
BGC_TD_dataset_10['sentence'] = BGC_TD_dataset_10['sentence'].apply(lambda x:' '.join(x))
BGC_TD_dataset_10['TDsentence'] = BGC_TD_dataset_10['TDsentence'].apply(lambda x:' '.join(x))
BGC_TD_dataset_10['TDlabels'] = BGC_TD_dataset_10['TDlabels'].apply(lambda x:[str(_) for _ in x]).apply(lambda x:' '.join(x))
BGC_TD_dataset_10

,ID,sentence,labels,isBGC,TDsentence,TDlabels
0,BGC0000436_0,Cluster_1153963 Cluster_1501678 Cluster_160734...,NRP,Yes,Cluster_1153963 Cluster_1501678 Cluster_160734...,1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
1,BGC0000436_1,Cluster_1153963 Cluster_1501678 Cluster_160734...,NRP,Yes,Cluster_0876677 Cluster_0442068 Cluster_093802...,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
2,BGC0000436_2,Cluster_1153963 Cluster_1501678 Cluster_160734...,NRP,Yes,EC:3.4.13.9 Cluster_0849311 Cluster_1331855 Cl...,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
3,BGC0000436_3,Cluster_1153963 Cluster_1501678 Cluster_160734...,NRP,Yes,Cluster_1629811 Cluster_1297778 Cluster_149144...,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
4,BGC0000436_4,Cluster_1153963 Cluster_1501678 Cluster_160734...,NRP,Yes,Cluster_1621985 Cluster_1595642 Cluster_157474...,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
...,...,...,...,...,...,...
55039,Negative_27517,Cluster_0610654 Cluster_1497579 Cluster_099825...,Unknown,No,Cluster_0610654 Cluster_1497579 Cluster_099825...,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
55040,Negative_27518,Cluster_0000170 Cluster_1620477 EC:3.2.1.196 E...,Unknown,No,Cluster_0000170 Cluster_1620477 EC:3.2.1.196 E...,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
55041,Negative_27519,Cluster_0525070 Cluster_0311313 Cluster_023474...,Unknown,No,Cluster_0525070 Cluster_0311313 Cluster_023474...,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
55042,Negative_27520,Cluster_0130209 Cluster_1286778 EC:3.4.11.4 EC...,Unknown,No,Cluster_0130209 Cluster_1286778 EC:3.4.11.4 EC...,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...


In [67]:
BGC_TD_dataset_10.to_csv('./BGC_TD_dataset_10.csv')

In [66]:
BGC_postive_dataset

,ID,sentence,labels,isBGC,TDsentence,TDlabels
0,BGC0000436_0,"[Cluster_1153963, Cluster_1501678, Cluster_160...",NRP,Yes,"[Cluster_1153963, Cluster_1501678, Cluster_160...","[1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,BGC0000436_1,"[Cluster_1153963, Cluster_1501678, Cluster_160...",NRP,Yes,"[Cluster_0876677, Cluster_0442068, Cluster_093...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,BGC0000436_2,"[Cluster_1153963, Cluster_1501678, Cluster_160...",NRP,Yes,"[EC:3.4.13.9, Cluster_0849311, Cluster_1331855...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,BGC0000436_3,"[Cluster_1153963, Cluster_1501678, Cluster_160...",NRP,Yes,"[Cluster_1629811, Cluster_1297778, Cluster_149...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,BGC0000436_4,"[Cluster_1153963, Cluster_1501678, Cluster_160...",NRP,Yes,"[Cluster_1621985, Cluster_1595642, Cluster_157...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...,...,...,...
27517,BGC0001595_6,"[Cluster_0923528, Cluster_1221717, Cluster_030...",Alkaloid,Yes,"[Cluster_1857686, Cluster_1652984, Cluster_023...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
27518,BGC0001595_7,"[Cluster_0923528, Cluster_1221717, Cluster_030...",Alkaloid,Yes,"[Cluster_1128388, Cluster_1603606, Cluster_003...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
27519,BGC0001595_8,"[Cluster_0923528, Cluster_1221717, Cluster_030...",Alkaloid,Yes,"[Cluster_0322584, Cluster_1434017, Cluster_114...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, ..."
27520,BGC0001595_9,"[Cluster_0923528, Cluster_1221717, Cluster_030...",Alkaloid,Yes,"[Cluster_1349909, Cluster_0931083, Cluster_017...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [70]:
BGC_TD_dataset_10

,ID,sentence,labels,isBGC,TDsentence,TDlabels
0,BGC0000436_0,Cluster_1153963 Cluster_1501678 Cluster_160734...,NRP,Yes,Cluster_1153963 Cluster_1501678 Cluster_160734...,1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
1,BGC0000436_1,Cluster_1153963 Cluster_1501678 Cluster_160734...,NRP,Yes,Cluster_0876677 Cluster_0442068 Cluster_093802...,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
2,BGC0000436_2,Cluster_1153963 Cluster_1501678 Cluster_160734...,NRP,Yes,EC:3.4.13.9 Cluster_0849311 Cluster_1331855 Cl...,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
3,BGC0000436_3,Cluster_1153963 Cluster_1501678 Cluster_160734...,NRP,Yes,Cluster_1629811 Cluster_1297778 Cluster_149144...,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
4,BGC0000436_4,Cluster_1153963 Cluster_1501678 Cluster_160734...,NRP,Yes,Cluster_1621985 Cluster_1595642 Cluster_157474...,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
...,...,...,...,...,...,...
55039,Negative_27517,Cluster_0610654 Cluster_1497579 Cluster_099825...,Unknown,No,Cluster_0610654 Cluster_1497579 Cluster_099825...,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
55040,Negative_27518,Cluster_0000170 Cluster_1620477 EC:3.2.1.196 E...,Unknown,No,Cluster_0000170 Cluster_1620477 EC:3.2.1.196 E...,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
55041,Negative_27519,Cluster_0525070 Cluster_0311313 Cluster_023474...,Unknown,No,Cluster_0525070 Cluster_0311313 Cluster_023474...,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
55042,Negative_27520,Cluster_0130209 Cluster_1286778 EC:3.4.11.4 EC...,Unknown,No,Cluster_0130209 Cluster_1286778 EC:3.4.11.4 EC...,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...


In [71]:
BGC_TD_dataset_5

,ID,sentence,labels,isBGC,TDsentence,TDlabels
0,BGC0000436_0,Cluster_1153963 Cluster_1501678 Cluster_160734...,NRP,Yes,Cluster_1153963 Cluster_1501678 Cluster_160734...,1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
1,BGC0000918_0,Cluster_0698230 EC:1.13.11.27 Cluster_1230425 ...,Other,Yes,Cluster_0698230 EC:1.13.11.27 Cluster_1230425 ...,1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
2,BGC0000740_0,Cluster_1607346 Cluster_0676609 Cluster_090172...,Saccharide,Yes,Cluster_1607346 Cluster_0676609 Cluster_090172...,1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
3,BGC0002659_0,Cluster_1042555 Cluster_0095569 Cluster_176389...,RiPP,Yes,Cluster_1042555 Cluster_0095569 Cluster_176389...,1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
4,BGC0001750_0,Cluster_0175250 Cluster_1313057 EC:6.2.1.3 Clu...,NRP Polyketide,Yes,Cluster_0175250 Cluster_1313057 EC:6.2.1.3 Clu...,1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 ...
...,...,...,...,...,...,...
30019,Negative_15007,Cluster_0511072 Cluster_0355445 Cluster_048772...,Unknown,No,Cluster_0511072 Cluster_0355445 Cluster_048772...,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
30020,Negative_15008,Cluster_1642682 Cluster_1280940 Cluster_176401...,Unknown,No,Cluster_1642682 Cluster_1280940 Cluster_176401...,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
30021,Negative_15009,Cluster_0408670 Cluster_1466465 Cluster_063779...,Unknown,No,Cluster_0408670 Cluster_1466465 Cluster_063779...,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
30022,Negative_15010,Cluster_1302719 Cluster_1374535 Cluster_130044...,Unknown,No,Cluster_1302719 Cluster_1374535 Cluster_130044...,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
